In [9]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import math
from framework import Framework

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

seed = 1295729
np.random.seed(seed=seed)
rng=np.random.default_rng(seed=seed)

In [10]:
### perm ansatz
# [1] Nicola Mariella, Andrea Simonetto, *A Quantum Algorithm for the Sub-Graph Isomorphism Problem*, https://arxiv.org/abs/2111.09732#
# [2] https://github.com/qiskit-community/subgraph-isomorphism

from typing import Union, Tuple
from qiskit.circuit import ParameterVector

IntOrTuple = Union[int, Tuple]

def _hcph(phi: float, qc: QuantumCircuit, qubits: IntOrTuple):
    ctrl, target = qubits if isinstance(qubits, tuple) else (-1, qubits)
    qc.h(target)
    if ctrl >= 0:
        qc.cp(phi, ctrl, target)
    else:
        qc.p(phi, target)
    qc.h(target)


S4_BLOCK_PARCOUNT = 5


def _s4_block(params: np.ndarray) -> QuantumCircuit:
    """Build the basic block S4 for the permutations Ansatz.

    Args:
        params (np.ndarray): The array of parameters.
    """
    params = np.asarray(params).flatten()
    assert params.shape == (S4_BLOCK_PARCOUNT,)
    qc = QuantumCircuit(QuantumRegister(name="q", size=2))
    _hcph(params[0], qc, 0)

    qc.h(1)
    qc.p(params[1], 1)
    qc.cp(params[2], 0, 1)
    qc.h(1)

    _hcph(params[3], qc, (1, 0))
    _hcph(params[4], qc, (0, 1))
    return qc


def _map_qreg(array, qreg: QuantumRegister) -> np.ndarray:
    assert isinstance(qreg, QuantumRegister)
    array = np.asarray(array)
    fun = np.vectorize(lambda i: qreg[i])
    return fun(array)


def _expand_topology(topology, *, qreg: QuantumRegister) -> np.ndarray:
    if isinstance(topology, str):
        if topology in {"linear", "circular"}:
            # v = np.arange(qreg.size - 1)
            # v = np.stack([v, v + 1]).T
            v = np.array([[qreg.size - 1, 0]])
            for i in range(qreg.size):
                for j in range(qreg.size):
                    if i < j:
                        v = np.concatenate([v, np.array([[i, j]])], axis=0)
            # if topology == "circular" and qreg.size > 2:
            #     v = np.concatenate([v, np.array([[qreg.size - 1, 0]])], axis=0)
            topology = v
        else:
            raise ValueError(f"Unrecognized topology: {topology}")
    topology = np.asarray(topology)
    assert topology.ndim == 2 and topology.shape[1] == 2
    return _map_qreg(topology, qreg=qreg)

def params_tensor(shape, *, name="t") -> np.ndarray:
    """Prepare a tensor of circuit parameters."""
    shape = tuple(np.atleast_1d(shape).flatten())
    v = ParameterVector(name=name, length=np.product(shape))
    v = np.array(v.params)
    return v.reshape(shape)

def s4_ansatz(
    topology, *, qreg: Union[QuantumRegister, int], params=None
) -> Tuple[QuantumCircuit, np.ndarray]:
    """Construct the permutations ansatz based on the S4 block.

    Args:
        topology (str, np.ndarray): The topology for the ansatz, see the function
        ansatz() for more information.
        qreg (QuantumRegister, int): The destination quantum register.
        params (np.ndarray): The array of parameters.
    """
    if isinstance(qreg, int):
        qreg = QuantumRegister(qreg)
    assert isinstance(qreg, QuantumRegister)
    topology = _expand_topology(topology, qreg=qreg)
    if params is None:
        params = params_tensor((len(topology), S4_BLOCK_PARCOUNT))
    params = np.asarray(params)
    assert params.ndim == 2 and params.shape[1] == S4_BLOCK_PARCOUNT
    assert len(params) == len(topology)
    qc = QuantumCircuit(qreg)
    for v, q in zip(params, topology):
        qc.compose(_s4_block(v), qubits=q, inplace=True)

    qc_ = QuantumCircuit(qreg)
    qc_.compose(qc.to_gate(label="PermAnsatz"), inplace=True)
    return qc_, params

def thetas_to_prob(x) -> np.ndarray:
    x = np.asarray(x) / np.pi
    x = np.abs(x)
    r = np.modf(x)
    r = r[0], r[1] % 2
    return np.abs(r[0] - r[1])

def sample_exact_thetas(v, *, n=1, seed=None):
    if isinstance(v, dict):
        dkeys = v.keys()
        v = np.array(list(v.values()))
    v = thetas_to_prob(v)
    rng = np.random.default_rng(seed)
    prob = rng.uniform(size=(n, len(v)))
    v = (prob < v) * np.pi
    if dkeys is not None:
        v = [dict(zip(dkeys, v1)) for v1 in v]
        assert len(v) == n
    return v

In [11]:
def Classic_Solve(adj):
    N = len(adj)

    M = 1 << (N - 1)

    dp = np.ones((N, M)) * np.inf
    dp[:, 0] = adj[:, 0]

    for j in range(1, M):
        for i in range(N):
            if i != 0 and (j >> (i - 1)) & 1 == 1:
                continue
            for k in range(1, N):
                if (j >> (k - 1)) & 1 != 1:
                    continue
                if dp[i][j] > adj[i][k] + dp[k][j ^ (1 << (k - 1))]:
                    dp[i][j] = adj[i][k] + dp[k][j ^ (1 << (k - 1))]

    suc = 0
    j = M - 1
    tour = [0]
    for i in range(N - 1):
        min_len = np.inf
        for k in range(1, N):
            if (j >> (k - 1)) & 1 != 1:
                continue
            if min_len > adj[suc][k] + dp[k][j ^ (1 << (k - 1))]:
                min_len = adj[suc][k] + dp[k][j ^ (1 << (k - 1))]
                tmp = k
        suc = tmp
        tour.append(suc)
        j = j ^ (1 << (suc - 1))

    return dp[0][M - 1], tour


In [12]:
def RandomPermutationMatrix(n):
  p = np.arange(n) + 1
  np.random.shuffle(p)
  m = []
  for i in range(n):
    line = np.zeros(n).astype(int)
    line[p[i] - 1] = 1
    m.append(line)
  return np.array(m)

In [13]:
# def RandomCompleteGraph(n):
#     g = nx.DiGraph()
#     for i in range(n):
#       for j in range(n):
#         if i != j:
#             g.add_edge(i, j, weight=np.random.randint(20) + 1)

#     adj_g = nx.adjacency_matrix(g).todense()
#     nx_ans, order = Classic_Solve(adj_g)
#     return adj_g, nx_ans, order

def RandomCompleteGraph(n, max_len):
    g = nx.random_geometric_graph(
        n, radius=0.4, seed=seed)
    pos = nx.get_node_attributes(g, "pos")

    pos[0] = (0.5, 0.5)

    for i in range(len(pos)):
        for j in range(i + 1, len(pos)):
            dist = math.hypot(pos[i][0] - pos[j][0], pos[i][1] - pos[j][1])
            dist = int(dist * max_len)
            g.add_edge(i, j, weight=dist)

    adj_g = nx.adjacency_matrix(g).todense()
    nx_ans, order = Classic_Solve(adj_g)
    
    return adj_g, nx_ans, order

In [14]:
def solveTSP(n, adj, get_info):
    qbit = int(np.log2(n))
    qc, params = s4_ansatz('circular', qreg=qbit)
    ansatz = QuantumCircuit(qbit * 2)
    ansatz.compose(qc, inplace=True, qubits=range(qbit))
    ansatz.compose(qc, inplace=True, qubits=np.array(range(qbit)) + qbit)

    a = np.zeros(n * n)
    for i in range(n-1):
        a[i * n + i + 1] = 1
    a[(n-1) * n] = 1

    f = Framework()
    f.encode(adj.flatten())
    f.operation(a)
    f.set_ansatz(ansatz)

    qubits_number = 0
    depth = 0
    if get_info:
        qc = f.loss_qc()
        qc = transpile(qc, basis_gates=['cx', 'u1', 'u2', 'u3'])
        qubits_number = len(qc.qubits)
        depth = qc.depth()

    min_cost = np.inf
    initial_point = (rng.uniform(size=len(qc.parameters)) - 1/2) * np.pi
    result = f.Run(seed=seed, init_point=initial_point)

    qc1 = s4_ansatz('circular', qreg=int(np.log2(len(adj))), params=params)[0]
    sampled_params_dicts = sample_exact_thetas(result.optimal_parameters,
                                                n=32, seed=seed)
    
    for v in sampled_params_dicts:
        p1 = np.abs(Operator(qc1.bind_parameters(v)).data)
        p1 = np.round(p1)
        cost = ((p1 @ adj @ p1.T) * a.reshape(n, n)).sum()
        if cost < min_cost:
            min_cost = cost
            p2 = p1
    
    return min_cost, p2, (qubits_number, depth)

In [15]:
def RandomCompleteGraphTest(n, RandomPermu=100, max_len=20):
    adj_g, nx_ans, _ = RandomCompleteGraph(n, max_len)

    all_cost = []
    for i in range(RandomPermu):
        P = RandomPermutationMatrix(n)
        adj = P @ adj_g @ P.T
        cost, p, info = solveTSP(n, adj, False)
        if i == 0:
            ret_info = info
        all_cost.append([cost, p.tolist()])

    return nx_ans, all_cost, adj_g, ret_info


In [16]:
ans, all_cost, adj_g, info = RandomCompleteGraphTest(8, 10, 20)

quantum_ans = np.array(all_cost)[:, 0]

print(adj_g)
print("answer: ", ans)
print("min: ", np.min(quantum_ans))
print("max: ", np.max(quantum_ans))
print("mean: ", np.mean(quantum_ans))
print("mean(100%): ", np.mean(np.abs(quantum_ans - ans)) / ans * 100 + 100)
print("info: ", info)

# with open('./TSP/16-complete-max300-test100-1', 'a+') as f:
#   print("seed = ", seed, file=f)
#   print(adj_g, file=f)
#   print("info: ", info, file=f)
#   print("answer: ", ans, file=f)
#   print("min: ", np.min(quantum_ans), file=f)
#   print("max: ", np.max(quantum_ans), file=f)
#   print("mean: ", np.mean(quantum_ans), file=f)
#   print("mean(100%): ", np.mean(np.abs(quantum_ans - ans)) / ans * 100 + 100, file=f)
#   print("quantum_ans: ", quantum_ans, file=f)
#   print(all_cost, file=f)

[[ 0  5  4  9  7  8 10  1]
 [ 5  0  9  4 11 13 15  3]
 [ 4  9  0 13 10  9 11  6]
 [ 9  4 13  0 13 16 17  7]
 [ 7 11 10 13  0  4  5  8]
 [ 8 13  9 16  4  0  2  9]
 [10 15 11 17  5  2  0 12]
 [ 1  3  6  7  8  9 12  0]]
answer:  41.0
min:  46.0
max:  56.0
mean:  50.2
mean(100%):  122.4390243902439
info:  (0, 0)


/var/folders/3b/54p896jd5hz9r642m7ngjx7h0000gn/T/ipykernel_2441/4137291175.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  quantum_ans = np.array(all_cost)[:, 0]
